In [30]:
import cv2
import mediapipe as mp
import keyboard 
import numpy as np
import json

#init
#---------------------
cap= cv2.VideoCapture(0)

hands= mp.solutions.hands.Hands(static_image_mode=False,
                        max_num_hands=1,
                        min_tracking_confidence=0.5,
                        min_detection_confidence=0.5)

mpDraw = mp.solutions.drawing_utils

#------------------------------
#variables 
savedGesturesFilePath="HandGestures.txt"
#basic quit function to close all windows
def quit(): 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        cap.release() 
        cv2.destroyAllWindows()
        return True    
#---------------------------------------
# reads the handcoordinates on the "r"-Key
def readCoords(init):
    if keyboard.is_pressed('r')|init:
        f = open(savedGesturesFilePath,"r")
        result= f.read()
        f.close()
        res = json.loads(result)
        return res
#---------------------------------------
#writes  the handcoordinates on the "space"-Key
def writeCoords():
     if keyboard.is_pressed('w'): 
        f = open(savedGesturesFilePath,"w")
        f.write(str(recognizedPoints)) 
        f.close()      
#---------------------------------------     
#compares the geestures, for the saved points and the points from the captured hand
def compareGestures(projectedPoint,pointFromArray):
   
    for x,y in enumerate(pointFromArray):
    #similarity is false at the start
        print(projectedPoint[0][0],pointFromArray[0])
        #compares x and y position of both arrays based on a threshold
        if(abs(projectedPoint[x][0]-pointFromArray[x][0])>0.07 or abs(projectedPoint[x][1]-pointFromArray[x][1])>0.07):
            #print("points from array",projectedPoint[x][1]-pointFromArray[x][1])
            return False
    #print if the points are similar for this threshold
        return True
#---------------------------------------   
# draw the standard hand from mediapipe
def drawHandMediaPipe(img, lm):
    h, w, _ = img.shape
    cx, cy = int(lm.x*w), int(lm.y*h)
    cv2.circle(img, (cx, cy), 3, (255, 0, 255)) 
    mpDraw.draw_landmarks(img, result.multi_hand_landmarks[0], mp.solutions.hands.HAND_CONNECTIONS)
#---------------------------------------   

def drawSavedHandGesture(id,h,w, recognizedColor):
     
     if len(savedCoords[0])<=2:
            return
     else:
     # get the hand wrist as compare offset x,y,z and merge them to the xyz
        pivotPointX = result.multi_hand_landmarks[0].landmark[1].x-savedCoords[0][0]
        pivotPointY = result.multi_hand_landmarks[0].landmark[1].y -savedCoords[0][1]
        pivotPointZ = result.multi_hand_landmarks[0].landmark[1].z -savedCoords[0][2]
        pivotXYZ = pivotPointX,pivotPointY,pivotPointZ
        #Draw the tracked handgesture as before but without the lines in between
        pivotTracked = int( result.multi_hand_landmarks[0].landmark[1].x*w), int( result.multi_hand_landmarks[0].landmark[1].y*h)
        # calculate the savedCoordinates with the offset of the current projected gesture to match the hand wrist position independed of the screen
        savedOffset =  [savedCoords[id][0]+pivotXYZ[0],  savedCoords[id][1]+pivotXYZ[1],  savedCoords[id][2]+pivotXYZ[2]]
        #for visual representation, it has to be redrawn in respect to the width and height ratio of the screen
        pivotOffsetImageRatio = int ((savedCoords[id][0]+pivotXYZ[0])*w), int ((savedCoords[id][1]+pivotXYZ[1])*h)
        #draw the circle for the tracked hand
        cv2.circle(img, (pivotTracked), 5, (255, 0, 0)) 
        #draw the circle for the saved matching hand gesture
        cv2.circle(img ,pivotOffsetImageRatio,5, recognizedColor)
   
        return pivotOffsetImageRatio
#Init params
recognizedPoints = np.array([0,0,0])
savedOffsetPoints= np.array([0,0,0])
savedCoords = readCoords(True)
recognizedColor = (0,0,255)

while True:    
    _,img = cap.read()
    result= hands.process(img)
   # if the gestrues should be compared this frame, start the compare method in the for loop
    if result.multi_hand_landmarks:
        recognizedPoints = []# clear the array before the loop starts to avoid overflow    
        savedOffsetPoints = []# clear the array before the loop starts to avoid overflow  
        h, w, _ = img.shape
        # for each recognized point on the hand, get the id and the bone coordinates
        for id, lm in enumerate(result.multi_hand_landmarks[0].landmark):   
            #calculate the saved handgesture in respect to the current hand position to make it independed of the image aspect
            savedOffsetPoints.append(drawSavedHandGesture(id,h,w, recognizedColor))
            #set the default points of the current handpostion
            recognizedPoints.append([lm.x, lm.y, lm.z])# append the circles from the hand to the array, 21 in total        
        # default hand drawing from puling
        drawHandMediaPipe(img, lm)
        # Draw the result
    cv2.imshow("Hand tracking test", img)

      # only if every comparepoint passed, set it to true, else break
      # check if there are enough gesturepoints to compare, before calling 
    recognizedColor =(255,0,0)
    if len(savedOffsetPoints)>=20 and compareGestures(recognizedPoints, savedOffsetPoints):
       #print("recognized!")
       #!!!Not Working 
       #If the gesture was recognized, color it green
       recognizedColor=(0,255,0)
    if quit():
        break

    writeCoords()
    readCoords(False)  
  
        


[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.8536989688873291, 0.26366037130355835, 6.054300456526107e-07] (497, 162)
[0.853698968